In [1]:
import nibabel as nib
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
import nilearn.image
import sklearn.decomposition
from sklearn.linear_model import Ridge
#from fMRI_Regions import atlas
from fMRI_Regions import atlas_masker
import sklearn.metrics as sm
from sampler import Sampler
from settings import settings
from scipy import signal
import scipy
import time
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Activation, Dropout, Lambda, Multiply, Add, Concatenate
from keras.optimizers import Adam
sampler = Sampler('/Users/ilamiheev/Desktop/data/')
num_frames = 300
num_train_frames = 210
num_test_frames = 60
train_start = settings.frame_creation_time * 5
train_end = settings.frame_creation_time * (num_train_frames + 5)
test_start = settings.frame_creation_time * (num_frames - num_test_frames)
test_end = settings.frame_creation_time * num_frames
patient_list = sampler.patient_list
current_patient = 35
random_state = 42

/Users/ilamiheev/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'keras'

In [ ]:
x_train, y_train, x_fl_train = sampler.create_one_man_dataset(patient=current_patient, start_time=train_start,
                                                                  end_time=train_end)

In [ ]:
x_test, y_test, x_fl_test =  sampler.create_one_man_dataset(patient=current_patient, start_time=test_start,
                                                               end_time=test_end)

In [ ]:
n_filters = 32 
filter_width = 2
dilation_rates = [2**i for i in range(8)] * 2 
history_seq = Input(shape=(None, 1))
x = history_seq
skips = []
for dilation_rate in dilation_rates:
    x = Conv2D(16, 1, padding='same', activation='relu')(x) 
    x_f = Conv2D(filters=n_filters,
                 kernel_size=filter_width, 
                 padding='causal',
                 dilation_rate=dilation_rate)(x)
    x_g = Conv2D(filters=n_filters,
                 kernel_size=filter_width, 
                 padding='causal',
                 dilation_rate=dilation_rate)(x)
    z = Multiply()([Activation('tanh')(x_f),
                    Activation('sigmoid')(x_g)])
    z = Conv2D(16, 1, padding='same', activation='relu')(z)
    x = Add()([x, z])    
    skips.append(z)  
out = Activation('relu')(Add()(skips))
out = Conv2D(128, 1, padding='same')(out)
out = Activation('relu')(out)
out = Dropout(.2)(out)
out = Conv2D(1, 1, padding='same')(out)

#def slice(x, seq_length):
    #return x[:,-seq_length:,:]

pred_seq_train = Lambda(slice, arguments={'seq_length':60})(out)
model = Model(history_seq, pred_seq_train)
model.compile(Adam(), loss='mean_absolute_error')